In [ ]:
import pandas as pd
import psycopg2
from psycopg2 import sql


In [ ]:
import psycopg2

db_config = {
    'host': 'localhost',
    'database': 'postgres',  # 先連到預設的 postgres DB
    'user': 'postgres_n',
    'password': 'postgres_p',  # 用你的密碼
    'port': 15433  # 注意：你的 port 是 15433，不是 5432
}

try:
    con = psycopg2.connect(**db_config)
    cursor = con.cursor()
    
    # 列出所有資料庫
    cursor.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
    databases = cursor.fetchall()
    
    print("可用的資料庫：")
    for db in databases:
        print(f"  - {db[0]}")
    
    cursor.close()
    con.close()
    
except Exception as e:
    print(f"錯誤：{e}")

In [ ]:
# PostgreSQL 連接設定
db_config = {
    'host': 'localhost',
    'database': 'medical_imaging',
    'user': 'postgres_n',
    'password': 'postgres_p',
    'port': 15433
}
con = None

try:
    # 連接到 PostgreSQL
    con = psycopg2.connect(**db_config)
    print(type(con))
    
    # 執行查詢並轉換為 DataFrame
    result = pd.read_sql(
        "SELECT * FROM one_page_text_report_v2 WHERE report_type = %s and content_raw like %s ",
        con,
        params=('system_data','%"image"%')
    )
    print(result)
    
finally:
    if con:
        con.close()

In [ ]:
result

In [ ]:
type(result['content_raw'].iloc()[4])

In [ ]:
result_dict_list = result.to_dict(orient='records')
result_dict_list[:2]

In [ ]:
import json
reault_dict = json.loads(result_dict_list[0]['content_raw'])['image']

In [ ]:
reault_dict

In [1]:
"""
完整版批量導入 - 同時處理 Report 和 ReportVersion
直接在 Jupyter 中使用
"""

import psycopg2
import psycopg2.extras
import pandas as pd
import json
import hashlib
from datetime import datetime
import uuid

db_config = {
    'host': 'localhost',
    'database': 'medical_imaging',
    'user': 'postgres_n',
    'password': 'postgres_p',
    'port': 15433
}

# ============ STEP 1: 獲取 system_data 記錄 ============

con = psycopg2.connect(**db_config)

query = """
    SELECT 
        uid,
        report_id,
        title,
        content_raw,
        verified_at,
        chr_no,
        mod,
        report_date,
        metadata
    FROM one_page_text_report_v2 
    WHERE report_type = 'system_data'
    AND  content_raw like '%"image"%'
    ORDER BY verified_at DESC
"""

df_system_data = pd.read_sql(query, con)
print(f"✓ 獲取了 {len(df_system_data)} 條 system_data 記錄")
print(df_system_data.head(2))



C:\Users\user\AppData\Local\Temp\ipykernel_99024\694645629.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_system_data = pd.read_sql(query, con)


✓ 獲取了 2447 條 system_data 記錄
                                uid report_id                   title   
0  191aa13d0c198129cca2c06f883ae38a      None  system_data - 11906793  \
1  0abdd630761da874bf186008020f5fc2      None  system_data - 00011046   

                                         content_raw   
0  {"image":[{"id":"11206290889","title":"Chest C...  \
1  {"image":[{"id":"100502773","title":"Lower Ext...   

                verified_at    chr_no              mod report_date   
0 2025-11-03 16:00:00+00:00  11906793  pt.get_resource  2025-11-04  \
1 2025-11-03 16:00:00+00:00  00011046  pt.get_resource  2025-11-04   

                                            metadata  
0  {'legacy_id': 'unknown', 'legacy_uid': '119067...  
1  {'legacy_id': 'unknown', 'legacy_uid': '000110...  


In [2]:
df_system_data.head()

,uid,report_id,title,content_raw,verified_at,chr_no,mod,report_date,metadata
0,191aa13d0c198129cca2c06f883ae38a,None,system_data - 11906793,"{""image"":[{""id"":""11206290889"",""title"":""Chest C...",2025-11-03 16:00:00+00:00,11906793,pt.get_resource,2025-11-04,"{'legacy_id': 'unknown', 'legacy_uid': '119067..."
1,0abdd630761da874bf186008020f5fc2,None,system_data - 00011046,"{""image"":[{""id"":""100502773"",""title"":""Lower Ext...",2025-11-03 16:00:00+00:00,00011046,pt.get_resource,2025-11-04,"{'legacy_id': 'unknown', 'legacy_uid': '000110..."
2,4a25c75be05635f27b4f12e4f66fc391,None,system_data - 00011046,"{""image"":[{""id"":""100219313"",""title"":""KUB"",""chr...",2025-11-03 16:00:00+00:00,00011046,pt.get_resource,2025-11-04,"{'legacy_id': 'unknown', 'legacy_uid': '000110..."
3,31100efc420a9132534740965f85a31b,None,system_data - 00012283,"{""image"":[{""id"":""100608338"",""title"":""Shoulder ...",2025-11-03 16:00:00+00:00,00012283,pt.get_resource,2025-11-04,"{'legacy_id': 'unknown', 'legacy_uid': '000122..."
4,2120cc8ff441b00658d2332cbc8a6ba0,None,system_data - 11912598,"{""image"":[{""id"":""11311240197"",""title"":""Chest C...",2025-11-03 16:00:00+00:00,11912598,pt.get_resource,2025-11-04,"{'legacy_id': 'unknown', 'legacy_uid': '119125..."


In [14]:
# ============ STEP 2: 提取並展平資料 ============

def extract_and_flatten(df):
    """從 system_data 中提取並展平所有項目"""
    extracted_records = []
    items_by_type = {}
    
    for idx, row in df.iterrows():
            # 解析 JSON content
            try:
                content_data = json.loads(row['content_raw']) if isinstance(row['content_raw'], str) else row['content_raw']
                content_data = content_data['image']
                if not isinstance(content_data, list):
                    continue
            except TypeError:
                print('content_data',content_data)
                print(f"✗ 提取行 {idx} 時出錯: TypeError")
                continue
            
            for row in content_data:
                try:
                    record = create_record(
                        row=row,
                    )
                    extracted_records.append(record)
                except Exception as e:
                    print('content_data',content_data)
                    print(f"✗ 提取行 {idx} 時出錯: {e}")
                    continue
                
        

    
    return extracted_records, items_by_type


def create_record(row,):
    """創建單條記錄，包含內容和版本信息"""
    content = row['content']
    content_hash = hashlib.md5(content.encode('utf-8')).hexdigest()
    uid = uuid.uuid4().hex 
    return {
        'uid': uid,
        'report_id': row['id'],
        'title':  row['title'],
        'content_raw': content,
        'content_processed': content,
        'content_hash': content_hash,
        'verified_at': row['v_date'],
        'chr_no': row['chr_no'],
        'mod': row['mod'],
        'report_date': row['date'],
        'metadata': {
            'extracted_at': datetime.now().isoformat()
        },
        'version_number': 1,
        'is_latest': True,
    }


extracted_records, items_by_type = extract_and_flatten(df_system_data)
print(f"\n✓ 提取了 {len(extracted_records)} 項目")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
len(extracted_records)

52102

In [16]:
extracted_records

[{'uid': '70188b8697d242ff85aa9cb22ee49561',
  'report_id': '11206290889',
  'title': 'Chest Chest PA',
  'content_raw': '部立雙和醫院  影像醫學部  CR報告\r\n病歷號碼 : 11906793       科    別 : OPD       檢查日期 : 20230704\r\n病患姓名 : 劉諺禎       性    別 : F       檢查時間 : 095936\r\n年    齡 : 31             主治醫師 : 蘇茗軒       開單日期 : 20230629\r\n認證日期 : 20230711       認證時間 : 104408\r\n檢查項目 :\r\nChest Chest PA\r\n\r\nSummary :\r\nS:\r\n5/17 pap: HSIL(CIN 3 ) at OPD   Travel:none,  Occupation:none,  \r\nContact:none,  Cluster:none\r\nO:\r\npap\r\n\r\nImaging findings :\r\nChest (PA) showed:\r\n> mild prominent lung markings at bilateral lower lung fields\r\n> normal heart size\r\n> clear bilateral costophrenic angles \r\n> normal appearing bony thorax\r\n古然文醫師(放診專醫字第0780號)',
  'content_processed': '部立雙和醫院  影像醫學部  CR報告\r\n病歷號碼 : 11906793       科    別 : OPD       檢查日期 : 20230704\r\n病患姓名 : 劉諺禎       性    別 : F       檢查時間 : 095936\r\n年    齡 : 31             主治醫師 : 蘇茗軒       開單日期 : 20230629\r\n認證日期 : 20230711       認證時間 : 10

In [22]:
def clean_null_bytes(text):
    """清理字符串中的 NULL 字符和其他不可見字符"""
    if not isinstance(text, str):
        return text
    # 移除 NULL 字符 (\x00) 和其他控制字符
    return text.replace('\x00', '').replace('\r\n', '\n')

# ============ STEP 3: 批量插入 Report 和 ReportVersion ============

def batch_insert_reports_and_versions(records, batch_size=500):
    """批量插入 Report 和 ReportVersion"""
    con = psycopg2.connect(**db_config)
    cursor = con.cursor()
    
    inserted_reports = 0
    inserted_versions = 0
    errors = 0
    
    total_batches = (len(records) + batch_size - 1) // batch_size
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, len(records))
        batch = records[start_idx:end_idx]
        
        try:
            # ============ 插入 Report ============
            report_query = """
                INSERT INTO one_page_text_report_v2 
                (uid, report_id, title, content_raw, content_processed, 
                 content_hash, source_url, verified_at, chr_no, mod, 
                 report_date, metadata, report_type, version_number, is_latest,
                 created_at, updated_at)
                VALUES 
                (%s, %s, %s, %s, %s, %s, '', %s, %s, %s, %s, %s::jsonb, 'image_extracted', %s, %s,
                 NOW(), NOW())
                ON CONFLICT (uid) DO UPDATE SET
                    content_raw = EXCLUDED.content_raw,
                    content_processed = EXCLUDED.content_processed,
                    content_hash = EXCLUDED.content_hash,
                    verified_at = EXCLUDED.verified_at,
                    version_number = EXCLUDED.version_number,
                    is_latest = EXCLUDED.is_latest,
                    updated_at = NOW()
            """
            
            for record in batch:
                try:
                    record['content_raw'] = clean_null_bytes(record['content_raw'])
                    record['content_processed'] = clean_null_bytes(record['content_processed'])
                    record['title'] = clean_null_bytes(record['title'])
                    cursor.execute(report_query, (
                        record['uid'],
                        record['report_id'],
                        record['title'][:500],
                        record['content_raw'],
                        record['content_processed'],
                        record['content_hash'],
                        record['verified_at'],
                        record['chr_no'],
                        record['mod'],
                        record['report_date'],
                        json.dumps(record['metadata']),
                        record['version_number'],
                        record['is_latest']
                    ))
                    inserted_reports += 1
                except Exception as e:
                    print(f"✗ 插入 Report 失敗: {e}")
                    errors += 1
                    con.rollback()
            
            con.commit()
            
            # ============ 插入 ReportVersion ============
            version_query = """
                INSERT INTO one_page_text_report_versions
                (report_id, version_number, content_hash, content_raw, 
                 verified_at, change_type, change_description, changed_at)
                VALUES 
                (%s, %s, %s, %s, %s, %s, %s, NOW())
                ON CONFLICT (report_id, version_number) DO UPDATE SET
                    content_hash = EXCLUDED.content_hash,
                    content_raw = EXCLUDED.content_raw,
                    verified_at = EXCLUDED.verified_at
            """
            
            for record in batch:
                try:
                    cursor.execute(version_query, (
                        record['uid'],
                        record['version_number'],
                        record['content_hash'],
                        record['content_raw'],
                        record['verified_at'],
                        'create',
                        'Image data extracted from system_data'
                    ))
                    inserted_versions += 1
                except Exception as e:
                    print(f"✗ 插入 ReportVersion 失敗: {e}")
                    errors += 1
                    con.rollback()
            
            con.commit()
            print(f"✓ 批次 {batch_num + 1}/{total_batches}: 插入 {len(batch)} 條 Report + ReportVersion")
            
        except Exception as e:
            print(f"✗ 批次 {batch_num + 1} 錯誤: {e}")
            errors += len(batch)
            con.rollback()
#         break
    
    cursor.close()
    con.close()
    return inserted_reports, inserted_versions, errors


print("開始批量插入...\n")
inserted_reports, inserted_versions, errors = batch_insert_reports_and_versions(
    extracted_records, 
    batch_size=500
)



開始批量插入...

✓ 批次 1/105: 插入 500 條 Report + ReportVersion
✓ 批次 2/105: 插入 500 條 Report + ReportVersion
✓ 批次 3/105: 插入 500 條 Report + ReportVersion
✓ 批次 4/105: 插入 500 條 Report + ReportVersion
✓ 批次 5/105: 插入 500 條 Report + ReportVersion
✓ 批次 6/105: 插入 500 條 Report + ReportVersion
✓ 批次 7/105: 插入 500 條 Report + ReportVersion
✓ 批次 8/105: 插入 500 條 Report + ReportVersion
✓ 批次 9/105: 插入 500 條 Report + ReportVersion
✓ 批次 10/105: 插入 500 條 Report + ReportVersion
✓ 批次 11/105: 插入 500 條 Report + ReportVersion
✓ 批次 12/105: 插入 500 條 Report + ReportVersion
✓ 批次 13/105: 插入 500 條 Report + ReportVersion
✓ 批次 14/105: 插入 500 條 Report + ReportVersion
✓ 批次 15/105: 插入 500 條 Report + ReportVersion
✓ 批次 16/105: 插入 500 條 Report + ReportVersion
✓ 批次 17/105: 插入 500 條 Report + ReportVersion
✓ 批次 18/105: 插入 500 條 Report + ReportVersion
✓ 批次 19/105: 插入 500 條 Report + ReportVersion
✓ 批次 20/105: 插入 500 條 Report + ReportVersion
✓ 批次 21/105: 插入 500 條 Report + ReportVersion
✓ 批次 22/105: 插入 500 條 Report + ReportVersion
✓ 批次 23/

In [ ]:

# ============ STEP 4: 統計報告 ============

print(f"""
╔════════════════════════════════════════════════════════╗
║       Image 數據導入完成統計報告                        ║
╚════════════════════════════════════════════════════════╝

原始系統數據記錄: {len(df_system_data)}
提取 image 項目總數: {items_count}

✓ Report 插入:
  - 成功: {inserted_reports}
  - 失敗: {errors}

✓ ReportVersion 插入:
  - 成功: {inserted_versions}
  - 失敗: {errors}

成功率: {(inserted_reports/len(extracted_records)*100):.2f}% (共 {len(extracted_records)} 條)

✓ 導入流程完成!

驗證查詢:
  1. 檢查插入的 Report 數量:
     SELECT COUNT(*) FROM one_page_text_report_v2 WHERE report_type = 'image_extracted';
  
  2. 檢查插入的 ReportVersion 數量:
     SELECT COUNT(*) FROM report_version WHERE change_description = 'Image data extracted from system_data';
  
  3. 查看一條完整記錄:
     SELECT r.uid, r.title, r.content_raw, rv.version_number 
     FROM one_page_text_report_v2 r
     LEFT JOIN report_version rv ON r.uid = rv.report_uid
     WHERE report_type = 'image_extracted'
     LIMIT 3;
""")

con.close()
